In [1]:
#|echo: false
import pandas as pd, numpy as np, matplotlib.pyplot as plt, altair as alt, pytz
from fastcore.all import *
from datetime import datetime, timedelta

## Get data

In [2]:
#|echo: false
print(f'Last execution time: {datetime.now(pytz.timezone("America/Lima")).strftime("%d/%m/%Y %T")}')

Last execution time: 22/06/2023 05:57:57


In [3]:
#|code-summary: Products type filter
explore_types = ['frutas', 'lacteos', 'verduras', 'embutidos', 'panaderia', 'desayuno', 'congelados', 'abarrotes',
                 'aves', 'carnes', 'pescados']

In [4]:
#|code-summary: Data table
path = Path('../../output')
csv_files = L(path.glob('*.csv')).filter(lambda o: os.stat(o).st_size>0)
pat_store = re.compile('(.+)\_\d+')
pat_date = re.compile('.+\_(\d+)')
df = (
    pd.concat([pd.read_csv(o).assign(store=pat_store.match(o.stem)[1], date=pat_date.match(o.stem)[1])
               for o in csv_files], ignore_index=True)
    .pipe(lambda d: d.assign(
        name=d.name.str.lower()+' ('+d.store+')',
        sku=d.id.where(d.sku.isna(), d.sku).astype(int),
        date=pd.to_datetime(d.date)
    ))
    .drop('id', axis=1)
    .loc[lambda d: d.category.str.contains('|'.join(explore_types))]
    # Filter products with recent data
    .loc[lambda d: d.name.isin(d.groupby('name').date.max().loc[ge(datetime.now()-timedelta(days=30))].index)]
    # Filter empty prices
    .loc[lambda d: d.price>0]
)
print(df.shape)
df.sample(3)

(506613, 8)


,brand,uri,name,price,category,store,date,sku
303528,SIBARITA,https://www.plazavea.com.pe/pimienta-molida-si...,pimienta molida sibarita sobre 21.6g bolsa 6un...,3.89,https://www.plazavea.com.pe/abarrotes,plaza_vea,2023-04-20,10028419
678980,Metro,https://www.metro.pe/jamon-pizza-metro-paquete...,jamón pizza metro paquete 180 g (metro),4.99,https://www.metro.pe/embutidos-y-fiambres/fiam...,metro,2023-01-09,558197
658213,PRIMOR,https://www.plazavea.com.pe/aceite-vegetal-pri...,aceite vegetal primor botella 1.8l (plaza_vea),20.90,https://www.plazavea.com.pe/abarrotes,plaza_vea,2023-04-03,10181308


## Top changes (ratio)

In [5]:
top_changes = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift())/d.shift())
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes.head(3)

,store,sku,last_price,change,last_date
8192,plaza_vea,10091709,12.99,0.799231,2023-06-22
2613,metro,564386,18.80,0.634783,2023-06-22
5751,plaza_vea,9493,5.49,0.347714,2023-06-22


In [6]:
def plot_changes(df_changes, title):
    selection = alt.selection_point(fields=['name'], bind='legend')
    dff = df_changes.drop('change', axis=1).merge(df, on=['store','sku'])
    return (dff
     .pipe(alt.Chart)
     .mark_line(point=True)
     .encode(
         x='date',
         y='price',
         color=alt.Color('name').scale(domain=sorted(dff.name.unique().tolist())),
         tooltip=['name','price','last_price']
     )
     .add_params(selection)
     .transform_filter(selection)
     .interactive()
     .properties(width=650, title=title)
     .configure_legend(orient='top', columns=3)
    )

In [7]:
top_changes.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [8]:
(top_changes
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [9]:
(top_changes
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Top changes (absolute values)

In [10]:
top_changes_abs = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift()).iloc[-1])
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes_abs.head(3)

,store,sku,last_price,change,last_date
6498,plaza_vea,25393,79.89,-25.01,2023-06-22
10204,plaza_vea,10906295,69.90,20.00,2023-06-22
7632,plaza_vea,10023353,29.90,18.00,2023-06-22


In [11]:
top_changes_abs.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [12]:
(top_changes_abs
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [13]:
(top_changes_abs
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Search specific products

In [14]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*pollo)(?=.*entero).*') &
                ~df.name.str.contains(r'marinado|aderezo')].unique().tolist()
names

['pollo entero light  x kg (metro)',
 'pollo entero fresco metro x kg (metro)',
 'pollo entero\xa0artisan\xa0libre de antibióticos x kg (plaza_vea)',
 'pollo entero sin menudencia x kg (metro)']

In [15]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Pollo')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [16]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'palta') &
                ~df.name.str.contains(r'shampoo|humectante|vino|salsa|acondicionador|aceite')].unique().tolist()
names

['palta hass x kg (metro)',
 'palta madura cremosita x kg (metro)',
 'palta fuerte metro x kg (metro)',
 'palta super fuerte x kg (metro)',
 'palta fuerte (plaza_vea)',
 'palta hass madura la caserita empaque 500g (plaza_vea)',
 "palta fuerte bell's madura (plaza_vea)",
 'palta nava metro x kg (metro)']

In [17]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Palta')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [18]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*aceite)(?=.*vegetal)(?=.*900).*') &
                ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')].unique().tolist()
names

['aceite vegetal cuisine&co botella 900 ml (metro)',
 'aceite vegetal máxima 900ml (metro)',
 'aceite vegetal primor clásico 900ml (metro)',
 'aceite vegetal deleite premium 900ml (metro)',
 'aceite vegetal primor premium 900ml (metro)',
 'aceite vegetal metro 900ml (metro)',
 'aceite vegetal cocinero 900ml (metro)',
 'aceite vegetal cil botella 900ml (plaza_vea)',
 'aceite vegetal nicolini botella 900ml (plaza_vea)',
 'aceite vegetal cocinero botella 900ml (plaza_vea)',
 'aceite vegetal primor premium botella 900ml (plaza_vea)',
 "aceite vegetal bell's botella 900ml (plaza_vea)",
 'aceite vegetal deleite botella 900ml (plaza_vea)',
 'aceite vegetal primor clásico botella 900ml (plaza_vea)',
 'aceite vegetal del cielo botella 900ml (plaza_vea)']

In [19]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [20]:
#|echo: false
#|output: false
names = df.name[
    df.name.str.contains(r'(?=.*harina)(?=.*1kg).*')
#     & ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')
].unique().tolist()
names

['harina sin preparar favorita 1kg (metro)',
 'harina preparada blanca flor 1kg (metro)',
 'harina preparada favorita 1kg (metro)',
 'harina sin preparar blanca flor 1kg (metro)',
 'harina preparada molitalia 1kg (metro)',
 'harina sin preparar molitalia 1kg (metro)',
 'harina sin preparar blanca flor bolsa 1kg (plaza_vea)',
 "harina bell's de trigo preparada bolsa 1kg (plaza_vea)",
 'harina selecta molitalia bolsa 1kg (plaza_vea)',
 'harina de trigo preparada blanca flor bolsa 1kg (plaza_vea)',
 'harina sin preparar favorita paquete 1kg (plaza_vea)',
 'harina preparada favorita paquete 1kg (plaza_vea)',
 'harina de maíz blanco p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina de arroz costeño bolsa 1kg (plaza_vea)',
 'harina de maíz amarillo p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina sin preparar grano de oro bolsa 1kg (plaza_vea)',
 'harina de trigo sin preparar molitalia sin preparar bolsa 1kg (plaza_vea)',
 'harina de trigo nicolini preparada bolsa 1kg (plaza_vea)',
 'harina

In [21]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)